## Importing Packages

In [1]:
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import itertools
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, recall_score,\
    accuracy_score, precision_score, f1_score, classification_report
import warnings
import xgboost

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('fake reviews dataset.csv')
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


## Data Understanding

In [3]:
df['category'].value_counts()

Kindle_Store_5                  4730
Books_5                         4370
Pet_Supplies_5                  4254
Home_and_Kitchen_5              4056
Electronics_5                   3988
Sports_and_Outdoors_5           3946
Tools_and_Home_Improvement_5    3858
Clothing_Shoes_and_Jewelry_5    3848
Toys_and_Games_5                3794
Movies_and_TV_5                 3588
Name: category, dtype: int64

In [4]:
df.describe()

,rating
count,40432.000000
mean,4.256579
std,1.144354
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [5]:
df['target'] = np.where(df['label'] == 'CG', 1, 0)

In [6]:
df

,category,rating,label,text_,target
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",1
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,1
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",1
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,1
...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...,0
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...,1
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ...",0
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...,1


## Text Preprocessing: Tokenization

In [7]:
#tokenizing the text data in the 'text_' column of df
def tokenizer(x):
    
    corpus = [word_tokenize(doc) for doc in x]

# getting common stop words in english that we'll remove during tokenization/text normalization
    stop_words = stopwords.words('english')
    corpus_no_stopwords = []
    for words in corpus:
        docs = [x.lower() for x in words if ((x.isalpha()) & (x not in stop_words))]
        corpus_no_stopwords.append(docs)
    return corpus_no_stopwords

## Lemmantizer

In [8]:
def lemmatizer(corpus, as_string=True):
    lem = WordNetLemmatizer()
    
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
    lemmatized_corpus = []
    for sentence in corpus:
        pos_tags = pos_tag(sentence)
        lemmatized_sentence = []
        for word, tag in pos_tags:
            pos = pos_tagger(tag)
            if pos is not None:
                lemmatized_word = lem.lemmatize(word, pos)
            else:
                lemmatized_word = lem.lemmatize(word)
            lemmatized_sentence.append(lemmatized_word)
        lemmatized_corpus.append(lemmatized_sentence)
    if as_string:
        lemmatized_corpus  = [' '.join(x) for x in lemmatized_corpus]
    return lemmatized_corpus
        

In [9]:
#corpus_tokenized = tokenizer(df['text_'])

In [10]:
#lemmatized_corpus = lemmatizer(corpus_tokenized)

## Pre vectorizing

In [11]:
# joined_lemm_corpus = [' '.join(x) for x in lemmatized_corpus]
# df['text_preproccesed'] = pd.Series(data=lemmatized_corpus)

In [12]:
vec = CountVectorizer(min_df = 0.05, max_df = 0.95)
X = vec.fit_transform(lemmatized_corpus)
countvec_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())

NameError: name 'lemmatized_corpus' is not defined

In [ ]:
tfidf = TfidfVectorizer(min_df = 0.05, max_df = 0.95)
Y = tfidf.fit_transform(lemmatized_corpus)
tfidf_df = pd.DataFrame(Y.toarray(), columns=tfidf.get_feature_names_out())

In [ ]:
tfidf_df

In [ ]:
X

In [13]:
X_train, X_test, y_train,y_test = train_test_split(df['text_'],df['target'], test_size=0.3,random_state=42)

In [32]:
X_train_preprocessed = lemmatizer(tokenizer(X_train),as_string=False)
X_test_preprocessed = lemmatizer(tokenizer(X_test),as_string=False)

### Try Random Forest and Logistic Regression with the Count Vectorizer

In [15]:
def scores(y_test, y_pred):
    print(f"""    Accuracy: {round(accuracy_score(y_test,y_pred),3)}
    
    Recall: {round(recall_score(y_test,y_pred),3)}
    
    F1: {round(f1_score(y_test,y_pred),3)}
    
    Precision: {round(precision_score(y_test,y_pred),3)}""")

In [16]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
pipe_cv_rf = Pipeline(steps)
pipe_cv_rf.fit(X_train_preprocessed, y_train)
pipe_cv_rf.score(X_train_preprocessed,y_train)

0.9825100699597201

In [17]:
crossval_rf_cv = np.mean(cross_val_score(pipe_cv_rf, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [18]:
crossval_rf_cv

0.7627021346850904

In [19]:
steps[1] = ('logreg',LogisticRegression(random_state=42))
pipe_cv_lr = Pipeline(steps)
pipe_cv_lr.fit(X_train_preprocessed, y_train)
pipe_cv_lr.score(X_train_preprocessed,y_train)

0.7288177513956611

In [20]:
crossval_lr_cv = np.mean(cross_val_score(pipe_cv_lr, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [21]:
crossval_lr_cv

0.7266269054338063

In [22]:
steps = [('tfidfvec',TfidfVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
pipe_idf_rf = Pipeline(steps)
pipe_idf_rf.fit(X_train_preprocessed, y_train)
pipe_idf_rf.score(X_train_preprocessed,y_train)

0.98187407250371

In [23]:
crossval_rf_idf = np.mean(cross_val_score(pipe_idf_rf, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [24]:
crossval_rf_idf

0.7631969029932032

In [25]:
steps[1] = ('logreg',LogisticRegression(random_state=42))
pipe_idf_lr = Pipeline(steps)
pipe_idf_lr.fit(X_train_preprocessed, y_train)
pipe_idf_lr.score(X_train_preprocessed,y_train)

0.7300190799236803

In [26]:
crossval_lr_idf = np.mean(cross_val_score(pipe_idf_lr, X_train_preprocessed,y_train,scoring='accuracy',cv=5))

In [27]:
crossval_lr_idf

0.7267683043675561

We run two models with both types of vectorizers, and based on our scores, we decided to stick with the Count Vectorizer.

In [28]:
steps_rf = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
steps_lr = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('logreg',LogisticRegression(random_state=42,max_iter=10000))]

pipe_rf = Pipeline(steps_rf)
pipe_lr = Pipeline(steps_lr)

In [29]:
param_rf = {'rfc__n_estimators':[100,150,200],
                       'rfc__max_depth':[2,3,4,5],
                       'rfc__min_samples_leaf':[1,2,3]
                      }

param_lr = {'logreg__penalty':['l1', 'l2', 'elasticnet', None],
                 'logreg__C':[0.001,0.01,0.1,1,10,100],
                 'logreg__solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
                }

In [31]:
gs_rf = GridSearchCV(estimator=pipe_rf, param_grid=param_rf,scoring='accuracy',cv=5,error_score=0)
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=param_lr,scoring='accuracy',cv=5,error_score=0)
gs_rf.fit(X_train_preprocessed,y_train)
gs_lr.fit(X_train_preprocessed,y_train)

GridSearchCV(cv=5, error_score=0,
             estimator=Pipeline(steps=[('countvec',
                                        CountVectorizer(max_df=0.95,
                                                        min_df=0.05)),
                                       ('logreg',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=42))]),
             param_grid={'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'logreg__penalty': ['l1', 'l2', 'elasticnet', None],
                         'logreg__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                            'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [ ]:
gs_rf.best_params_

In [ ]:
gs_lr.best_params_

In [ ]:
gs_rf.best_score_

In [ ]:
gs_lr.best_score_

In [ ]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('ada',AdaBoostClassifier())]
pipe_ada_booster = Pipeline(steps).fit(X_train_preprocessed,y_train)
pipe_ada_booster.score(X_train_preprocessed,y_train)

In [ ]:
crossval_ada_booster = np.mean(cross_val_score(pipe_ada_booster, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_ada_booster

In [ ]:
gbc = ('gbc',GradientBoostingClassifier())
steps[1] = gbc
pipe_gbc = Pipeline(steps).fit(X_train_preprocessed,y_train)
pipe_gbc.score(X_train_preprocessed,y_train)

In [ ]:
crossval_gradient_booster = np.mean(cross_val_score(pipe_gbc, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_gradient_booster

In [ ]:
param_ada_booster

### XGBClassifier with CountVectorizer

In [ ]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('xgb',xgboost.XGBClassifier(random_state=42, objective='binary:logistic'))]
xgb_pipe = Pipeline(steps).fit(X_train_preprocessed,y_train)
xgb_pipe.score(X_train_preprocessed, y_train)

In [ ]:
crossval_xg_booster = np.mean(cross_val_score(xgb_pipe, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_xg_booster

### Hyperparameter Grids for Boosting Algorithms

In [ ]:
param_grid_ada = {'ada__n_estimators':[50,100,150,200],
                  'ada__learning_rate':[0.01,0.1,0.2,0.5,1]
                 }

param_grid_gradient = {'gbc__n_estimators':[50,100,150,200],
                       'gbc__learning_rate':[0.01,0.1,0.2,0.5,1],
                       'gbc__max_depth':[1,2,3,4]
                      }

param_grid_xgb = {
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__max_depth': [3, 5, 7],
    'xgb__subsample': [0.8, 0.9, 1.0],
    'xgb__gamma': [0, 0.1, 0.2],
    'xgb__reg_lambda': [1, 1.5, 2]
}

In [ ]:
gs_ada = GridSearchCV(estimator=pipe_ada_booster, param_grid=param_grid_ada,scoring='accuracy',cv=5,error_score=0)
gs_gradient = GridSearchCV(estimator=pipe_gbc, param_grid=param_grid_gradient,scoring='accuracy',cv=5,error_score=0)
gs_xgb = GridSearchCV(estimator=xgb_pipe, param_grid=param_grid_xgb,scoring='accuracy',cv=5,error_score=0)

gs_ada.fit(X_train_preprocessed,y_train)

In [ ]:
gs_gradient.fit(X_train_preprocessed,y_train)

In [ ]:
gs_xgb.fit(X_train_preprocessed,y_train)

In [ ]:
# Best Hyperparameters for AdaBoost Classifier
gs_ada.best_params_

In [ ]:
# Best Cross-Validation Score for AdaBoost Classifier
gs_ada.best_score_

In [ ]:
# Best Hyperparameters for GradientBoosting Classifier
gs_gradient.best_params_

In [ ]:
# Best Cross-Validation Score for GradientBoosting Classifier
gs_gradient.best_score_

In [ ]:
# Best Hyperparameters for XGBClassifier 
gs_xgb.best_params_

In [ ]:
# Best Cross-Validation Score for XGBClassifier 
gs_xgb.best_score_